In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# In this notebook I change the daily time series to a weekly sum, merge the soil data of each location, and save the new csv for feature extraction plus modeling. This is because the score was only given out once a week and that is what we want to predict with the data.


# METRIC | DESCRIPTION
- fips | US County FIPS Codes
- date | Date Measurement is taken 
- PRECTOT | Precipitation (mm day-1)
- PS | Surface Pressure (kPa) 
- QV2M | Specific Humidity at 2 Meters (g/kg)
- T2M | Temperature at 2 Meters (C)
- T2MDEW | Dew/Frost Point at 2 Meters (C)
- T2MWET | Wet Bulb Temperature at 2 Meters (C)
- T2M_MAX | Maximum Temperature at 2 Meters (C)
- T2M_MIN | Minimum Temperature at 2 Meters (C)
- T2M_RANGE | Temperature Range at 2 Meters (C)
- TS | Earth Skin Temperature (C)
- WS10M | Wind Speed at 10 Meters (m/s) 
- WS10M_MAX | Maximum Wind Speed at 10 Meters (m/s)
- WS10M_MIN | Minimum Wind Speed at 10 Meters (m/s)
- WS10M_RANGE | Wind Speed Range at 10 Meters (m/s) 
- WS50M | Wind Speed at 50 Meters (m/s)
- WS50M_MAX | Maximum Wind Speed at 50 Meters (m/s)
- WS50M_MIN | Minimum Wind Speed at 50 Meters (m/s)
- WS50M_RANGE | Wind Speed Range at 50 Meters (m/s)
- score | Score of drought


In [2]:
metric_descrip = {'fips':'US County FIPS Codes', 'date':'Date Measurement is taken',
                  'PRECTOT':'Precipitation (mm day-1)', 'PS':'Surface Pressure (kPa)',
                  'QV2M':'Specific Humidity at 2 Meters (g/kg)',
                  'T2M':'Temperature at 2 Meters (C)', 'T2MDEW':'Dew/Frost Point at 2 Meters (C)', 
                  'T2MWET':'Wet Bulb Temperature at 2 Meters (C)', 'T2M_MAX':'Maximum Temperature at 2 Meters (C)',
                  'T2M_MIN':'Minimum Temperature at 2 Meters (C)', 'T2M_RANGE':'Temperature Range at 2 Meters (C)',
                  'TS':'Earth Skin Temperature (C)', 'WS10M':'Wind Speed at 10 Meters (m/s)', 
                  'WS10M_MAX':'Maximum Wind Speed at 10 Meters (m/s)', 'WS10M_MIN':'Minimum Wind Speed at 10 Meters (m/s)', 
                  'WS10M_RANGE':'Wind Speed Range at 10 Meters (m/s)', 'WS50M':'Wind Speed at 50 Meters (m/s)', 
                  'WS50M_MAX':'Maximum Wind Speed at 50 Meters (m/s)', 'WS50M_MIN':'Minimum Wind Speed at 50 Meters (m/s)',
                  'WS50M_RANGE':'Wind Speed Range at 50 Meters (m/s)', 'score':'Score of drought'}

In [3]:
# load all the data

In [4]:
%%time
df_train_daily = pd.read_csv('data/train_timeseries/train_timeseries.csv')
df_train_daily.head()

CPU times: user 38 s, sys: 16.7 s, total: 54.8 s
Wall time: 1min 19s


,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,...,14.65,2.20,2.94,1.49,1.46,4.85,6.04,3.23,2.81,NaN
1,1001,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,...,16.60,2.52,3.43,1.83,1.60,5.33,6.13,3.72,2.41,NaN
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,...,18.41,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,...,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,...,2.65,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN


In [5]:
df_test_daily = pd.read_csv('data/test_timeseries/test_timeseries.csv')
df_test_daily.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2019-01-01,2.25,100.51,9.69,14.71,13.55,13.52,17.38,10.92,...,14.63,1.20,1.50,0.79,0.71,2.74,4.01,1.23,2.78,0.0
1,1001,2019-01-02,4.94,100.48,8.65,13.05,11.83,11.74,17.76,9.54,...,13.06,1.02,1.35,0.32,1.03,2.13,3.37,0.56,2.81,NaN
2,1001,2019-01-03,20.74,100.03,8.59,12.12,11.67,11.67,13.74,10.44,...,12.12,1.83,4.23,0.34,3.88,3.41,7.49,0.72,6.77,NaN
3,1001,2019-01-04,16.17,99.47,7.93,11.80,10.38,9.59,18.19,5.44,...,11.72,3.75,5.25,2.34,2.91,6.78,9.61,4.90,4.71,NaN
4,1001,2019-01-05,0.01,100.39,5.32,7.91,4.73,4.71,14.37,3.68,...,7.75,2.32,2.90,1.69,1.21,4.42,5.87,3.15,2.72,NaN


In [6]:
df_val_daily = pd.read_csv('data/validation_timeseries/validation_timeseries.csv')
df_val_daily.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2017-01-01,32.50,100.02,10.47,14.69,14.47,14.47,17.68,10.53,...,14.63,2.14,2.71,1.52,1.19,4.40,5.96,2.25,3.71,NaN
1,1001,2017-01-02,63.52,100.04,12.75,17.96,17.75,17.75,20.30,16.14,...,17.85,2.75,4.31,1.60,2.71,5.50,8.16,4.05,4.11,NaN
2,1001,2017-01-03,18.82,99.69,9.74,14.24,13.44,13.44,18.48,9.29,...,14.06,2.25,3.73,1.64,2.09,4.80,7.27,2.54,4.72,2.0
3,1001,2017-01-04,0.01,100.02,5.21,8.10,3.86,3.88,11.74,2.12,...,8.08,2.63,3.95,1.34,2.60,4.98,6.16,3.36,2.80,NaN
4,1001,2017-01-05,0.01,99.89,4.54,5.91,2.20,2.22,13.07,-0.18,...,5.85,1.76,2.76,0.47,2.28,3.43,4.70,0.66,4.04,NaN


In [7]:
df_train_daily.columns

Index(['fips', 'date', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
       'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'score'],
      dtype='object')

In [8]:
# scores are not balanced, Will need to take weekly average senese score is only given on weekends
df_train_daily['score'].value_counts(dropna=False)

NaN       16543884
0.0000     1480827
1.0000      219135
2.0000      123789
3.0000       82801
            ...   
0.1145           1
2.0172           1
0.6750           1
1.3998           1
0.6060           1
Name: score, Length: 55396, dtype: int64

In [9]:
# check for unique values in all of the data and see if I need to get rid of duplicate data
def unique_values_for_df(data):
    size_df = len(data)
    print(size_df)
    for column in data.columns:
        u = len(data[column].unique())
        print(column + ': ', u, u/size_df)
        
unique_values_for_df(df_train_daily)

19300680
fips:  3108 0.00016103059581320451
date:  6210 0.00032175032175032174
PRECTOT:  10073 0.0005218987103045074
PS:  3728 0.00019315381634222215
QV2M:  2210 0.0001145037376921435
T2M:  7206 0.00037335472118080816
T2MDEW:  6022 0.00031200973229958737
T2MWET:  5982 0.0003099372664590056
T2M_MAX:  7513 0.0003892608965072733
T2M_MIN:  7052 0.00036537572769456826
T2M_RANGE:  3029 0.0001569374757780555
TS:  7588 0.00039314676995836414
WS10M:  1740 9.015226406530754e-05
WS10M_MAX:  2206 0.00011429649110808532
WS10M_MIN:  1450 7.512688672108962e-05
WS10M_RANGE:  1828 9.471168891458747e-05
WS50M:  2122 0.00010994431284286357
WS50M_MAX:  2664 0.00013802622498274673
WS50M_MIN:  1772 9.181023673777297e-05
WS50M_RANGE:  2186 0.00011326025818779442
score:  55396 0.0028701579426217106


In [10]:
# test data
df_test_daily['score'].value_counts(dropna=False)

NaN       1945608
0.0000     222634
1.0000      12196
2.0000       5585
3.0000       1875
           ...   
0.3978          1
2.3534          1
2.1108          1
0.2738          1
0.5248          1
Name: score, Length: 24421, dtype: int64

In [11]:
unique_values_for_df(df_test_daily)

2271948
fips:  3108 0.0013679890560875513
date:  731 0.00032175032175032174
PRECTOT:  7020 0.003089859451008562
PS:  3636 0.0016003887412916141
QV2M:  2205 0.0009705327762783303
T2M:  6805 0.0029952270034349377
T2MDEW:  6028 0.00265322973941305
T2MWET:  5766 0.002537910198648913
T2M_MAX:  7122 0.003134754844741165
T2M_MIN:  6796 0.002991265645164414
T2M_RANGE:  2666 0.001173442349912938
TS:  7156 0.003149719975985366
WS10M:  1493 0.0006571453219880032
WS10M_MAX:  1950 0.0008582942919468227
WS10M_MIN:  1189 0.0005233394426280883
WS10M_RANGE:  1625 0.0007152452432890189
WS50M:  1856 0.0008169201055657964
WS50M_MAX:  2377 0.0010462387343372296
WS50M_MIN:  1485 0.0006536241146364265
WS50M_RANGE:  1937 0.0008525723300005105
score:  24421 0.01074892559160685


In [12]:
# val data
df_val_daily['score'].value_counts(dropna=False)

NaN       1945608
0.0000     191773
1.0000      20325
2.0000       8666
3.0000       3335
           ...   
2.4107          1
3.3495          1
3.4205          1
3.4620          1
2.0155          1
Name: score, Length: 25248, dtype: int64

In [13]:
unique_values_for_df(df_val_daily)

2268840
fips:  3108 0.0013698630136986301
date:  730 0.00032175032175032174
PRECTOT:  7296 0.0032157402020415718
PS:  3645 0.0016065478394245518
QV2M:  2105 0.0009277868866909962
T2M:  6500 0.0028649001251740977
T2MDEW:  5724 0.002522875125614852
T2MWET:  5472 0.002411805151531179
T2M_MAX:  6918 0.003049135240916063
T2M_MIN:  6351 0.002799227799227799
T2M_RANGE:  2821 0.0012433666543255585
TS:  6912 0.0030464907177235943
WS10M:  1538 0.0006778794450027326
WS10M_MAX:  1929 0.00085021420637859
WS10M_MIN:  1295 0.0005707762557077625
WS10M_RANGE:  1575 0.0006941873380229544
WS50M:  1877 0.0008272950053771972
WS50M_MAX:  2338 0.0010304825373318525
WS50M_MIN:  1594 0.000702561661465771
WS50M_RANGE:  1886 0.0008312617901658997
score:  25248 0.011128153593907019


In [14]:
# make function for whole code 
def daily_to_weekly(data):
    df_weekly = {'fips':[], 'date':[], 'PRECTOT':[], 'PS':[], 'QV2M':[], 'T2M':[], 'T2MDEW':[], 
                      'T2MWET':[], 'T2M_MAX':[], 'T2M_MIN':[], 'T2M_RANGE':[], 'TS':[], 'WS10M':[], 
                      'WS10M_MAX':[],'WS10M_MIN':[],'WS10M_RANGE':[], 'WS50M':[],'WS50M_MAX':[], 'WS50M_MIN':[],
                      'WS50M_RANGE':[], 'score':[]}
    start_fips, end_fips, PRECTOT, PS, QV2M, T2M, T2MDEW, T2MWET, T2M_MAX = 0,0,0,0,0,0,0,0,0
    T2M_MIN, T2M_RANGE, TS, WS10M, WS10M_MAX, WS10M_MIN, WS10M_RANGE, WS50M = 0,0,0,0,0,0,0,0
    WS50M_MAX, WS50M_MIN, WS50M_RANGE, score, count  = 0,0,0,0,0
    start_date, end_date = '',''
    for i in data.iloc: 
        if count == 0:
            start_fips = i['fips']
            start_date = i['date']
        PRECTOT += i['PRECTOT']
        PS += i['PS']
        QV2M += i['QV2M']
        T2M += i['T2M']
        T2MDEW += i['T2MDEW']
        T2MWET += i['T2MWET']
        T2M_MAX += i['T2M_MAX']
        T2M_MIN += i['T2M_MIN']
        T2M_RANGE += i['T2M_RANGE']
        TS += i['TS']
        WS10M += i['WS10M']
        WS10M_MAX += i['WS10M_MAX']
        WS10M_MIN += i['WS10M_MIN']
        WS10M_RANGE += i['WS10M_RANGE']
        WS50M += i['WS50M']
        WS50M_MAX += i['WS50M_MAX']
        WS50M_MIN += i['WS50M_MIN']
        WS50M_RANGE += i['WS50M_RANGE']
        end_fips = i['fips']
        count += 1
        # need list for all the data/however to make new dataframe 
        # when condition is true do: (mean of all previous data for the week)
        if not np.isnan(i['score']) and count == 7:
            # append to dataframe when this happens 
            end_date = i['date']
            df_weekly['fips'].append(end_fips)
            # change date with the interval that will be there
            df_weekly['date'].append(start_date + '|' + end_date)
            df_weekly['PRECTOT'].append(PRECTOT)
            df_weekly['PS'].append(PS)
            df_weekly['QV2M'].append(QV2M)
            df_weekly['T2M'].append(T2M)
            df_weekly['T2MDEW'].append(T2MDEW)
            df_weekly['T2MWET'].append(T2MWET)
            df_weekly['T2M_MAX'].append(T2M_MAX)
            df_weekly['T2M_MIN'].append(T2M_MIN)
            df_weekly['T2M_RANGE'].append(T2M_RANGE)
            df_weekly['TS'].append(TS)
            df_weekly['WS10M'].append(WS10M)
            df_weekly['WS10M_MAX'].append(WS10M_MAX)
            df_weekly['WS10M_MIN'].append(WS10M_MIN)
            df_weekly['WS10M_RANGE'].append(WS10M_RANGE)
            df_weekly['WS50M'].append(WS50M)
            df_weekly['WS50M_MAX'].append(WS50M_MAX)
            df_weekly['WS50M_MIN'].append(WS50M_MIN)
            df_weekly['WS50M_RANGE'].append(WS50M_RANGE)
            # round score if not integer
            
            df_weekly['score'].append(round(i['score']))


            PRECTOT = 0
            PS = 0
            QV2M = 0
            T2M = 0
            T2MDEW = 0
            T2MWET = 0
            T2M_MAX = 0
            T2M_MIN = 0
            T2M_RANGE = 0
            TS = 0
            WS10M = 0
            WS10M_MAX = 0
            WS10M_MIN = 0
            WS10M_RANGE = 0
            WS50M = 0
            WS50M_MAX = 0
            WS50M_MIN = 0
            WS50M_RANGE = 0
            count = 0
        elif (not np.isnan(i['score']) and count != 7) or start_fips != end_fips:
            # skipped because not full week
            PRECTOT = 0
            PS = 0
            QV2M = 0
            T2M = 0
            T2MDEW = 0
            T2MWET = 0
            T2M_MAX = 0
            T2M_MIN = 0
            T2M_RANGE = 0
            TS = 0
            WS10M = 0
            WS10M_MAX = 0
            WS10M_MIN = 0
            WS10M_RANGE = 0
            WS50M = 0
            WS50M_MAX = 0
            WS50M_MIN = 0
            WS50M_RANGE = 0
            count = 0
       
    df_weekly = pd.DataFrame(data=df_weekly)
    df_weekly.head()
    return df_weekly

In [15]:
%%time
df_train_weekly = daily_to_weekly(df_train_daily)

CPU times: user 1h 14min 25s, sys: 27.1 s, total: 1h 14min 52s
Wall time: 1h 15min 32s


In [16]:
%%time
df_test_weekly = daily_to_weekly(df_test_daily)

CPU times: user 8min 39s, sys: 2.81 s, total: 8min 42s
Wall time: 8min 44s


In [17]:
%%time
df_val_weekly = daily_to_weekly(df_val_daily)

CPU times: user 8min 28s, sys: 2.11 s, total: 8min 30s
Wall time: 8min 32s


In [18]:
size_df_train_weekly = len(df_train_weekly)
print(size_df_train_weekly)
for column in df_train_weekly.columns:
    u = len(df_train_weekly[column].unique())
    print(column + ': ', u, u/size_df_train_weekly)

2753688
fips:  3108 0.001128668171557562
date:  886 0.00032175032175032174
PRECTOT:  42190 0.015321270964611822
PS:  58634 0.021292898832402218
QV2M:  43780 0.01589867842689513
T2M:  118421 0.043004508862296675
T2MDEW:  110562 0.040150518141488796
T2MWET:  109792 0.03987089314403084
T2M_MAX:  122592 0.04451920479008515
T2M_MIN:  117679 0.04273505204656446
T2M_RANGE:  42614 0.015475246287887372
TS:  122385 0.044444032875184114
WS10M:  19384 0.007039286948993495
WS10M_MAX:  25126 0.00912449050146567
WS10M_MIN:  14836 0.005387683717254823
WS10M_RANGE:  19416 0.007050907728108631
WS50M:  22645 0.0082235169706953
WS50M_MAX:  28482 0.010343219711165535
WS50M_MIN:  19442 0.007060349611139679
WS50M_RANGE:  21185 0.007693318923567231
score:  6 2.178896084087958e-06


In [19]:
# scores are all rounded for model
df_train_weekly['score'].value_counts(dropna=False)

0    1651128
1     466118
2     294563
3     196390
4     106265
5      39224
Name: score, dtype: int64

In [20]:
print(len(df_train_weekly))
df_train_weekly.drop_duplicates(['PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
       'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'score'])
print(len(df_train_weekly))

2753688
2753688


# no duplicate rows in df_train weekly

In [21]:
# add soil data 

# METRIC | DESCRIPTION
- fips | US County FIPS Codes
- lat | latitude 
- lon | longitude
- elivation | median elevation (meters) 
- slope1 | 0% <= slope <= 0.5% (percentage of slope in the fips area)
- slope2 | .5% <= slope <= 2% (percentage of slope in the fips area)
- slope3 | 2% <= slope <= 5% (percentage of slope in the fips area)
- slope4 | 5% <= slope <= 10% (percentage of slope in the fips area)
- slope5 | 10% <= slope <= 15% (percentage of slope in the fips area)
- slope6 | 15% <= slope <= 30% (percentage of slope in the fips area)
- slope7 | 30% <= slope <= 45% (percentage of slope in the fips area)
- slope8 | 45% < slope (percentage of slope in the fips area)
- aspectN| North: 0˚< aspect ≤45˚ or 315˚< aspect ≤360˚
- aspectE | East: 45˚ < aspect ≤ 135˚
- aspectS | South: 135˚ < aspect ≤ 225˚
- aspectW | West: 225˚ < aspect ≤ 315˚
- aspectUnknown | Undefined: Slope aspect undefined; this value is used for grids where slope gradient is undefined or slope 
- WAT_LAND | mapped water bodies (percenatage that adds to 1 for total amount of land)
- NVG_LAND | barren/very sparsely vegetated land (percenatage that adds to 1 for total amount of land)
- URB_LAND | built-up land (residential and infrastructure) (percenatage that adds to 1 for total amount of land)
- GRS_LAND | grass/scrub/woodland (percenatage that adds to 1 for total amount of land)
- FOR_LAND | forest land, calibrated to FRA2000 land statistics (percenatage that adds to 1 for total amount of land)
- CULTRF_LAND | rain-fed cultivated land (percenatage that adds to 1 for total amount of land)
- CULTIR_LAND | irrigated cultivated land (percenatage that adds to 1 for total amount of land)
- CULT_LAND | total cultivated land
- SQ1 | Nutrient availability
- SQ2 | Nutrient retention capacity
- SQ3 | Rooting conditions
- SQ4 | Oxygen availability to roots
- SQ5 | Excess salts.
- SQ6 | Toxicity
- SQ7 | Workability (constraining field management)

In [22]:
df_soil = pd.read_csv('data/soil_data.csv')
df_soil.head()

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1005,31.870670,-85.405456,146,0.0158,0.1868,0.5441,0.2424,0.0106,0.0003,...,72.578804,1.828159,74.406960,3,2,1,1,1,1,1
2,1003,30.659218,-87.746067,52,0.0746,0.4370,0.4415,0.0469,0.0000,0.0000,...,59.843639,2.996914,62.840553,3,2,1,2,1,1,1
3,1007,33.015893,-87.127148,93,0.0144,0.1617,0.3714,0.3493,0.0898,0.0134,...,1.916593,0.008330,1.924924,3,2,1,1,1,1,1
4,1009,33.977448,-86.567246,198,0.0050,0.0872,0.2799,0.3576,0.1477,0.1037,...,1.891909,0.027488,1.919397,3,2,1,1,1,1,1


In [23]:
# join soil data with weekly data by fips 
df_soil_train_weekly = pd.merge(df_soil, df_train_weekly, on=['fips'])
print(df_soil_train_weekly.columns)
df_soil_train_weekly.head()

Index(['fips', 'lat', 'lon', 'elevation', 'slope1', 'slope2', 'slope3',
       'slope4', 'slope5', 'slope6', 'slope7', 'slope8', 'aspectN', 'aspectE',
       'aspectS', 'aspectW', 'aspectUnknown', 'WAT_LAND', 'NVG_LAND',
       'URB_LAND', 'GRS_LAND', 'FOR_LAND', 'CULTRF_LAND', 'CULTIR_LAND',
       'CULT_LAND', 'SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6', 'SQ7', 'date',
       'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET', 'T2M_MAX',
       'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN',
       'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'WS50M_RANGE',
       'score'],
      dtype='object')


,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,59.91,15.40,23.14,8.67,14.46,30.93,46.12,15.96,30.15,2
1,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,69.83,16.47,22.80,10.43,12.37,34.09,45.56,19.69,25.85,2
2,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,31.56,20.53,28.63,12.70,15.93,36.89,50.22,25.03,25.20,2
3,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,5.47,16.54,23.48,9.23,14.26,30.61,43.24,18.06,25.16,1
4,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,25.48,14.53,21.24,7.36,13.88,26.69,41.16,11.58,29.59,1


In [24]:
df_soil_test_weekly = pd.merge(df_soil, df_test_weekly, on=['fips'])
df_soil_test_weekly.head()

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,77.09,14.56,21.70,7.87,13.82,28.53,43.71,15.21,28.50,0
1,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,39.22,16.61,22.50,12.03,10.47,31.96,41.70,23.81,17.90,0
2,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,42.73,18.51,26.27,10.28,16.00,33.80,49.32,20.19,29.13,0
3,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,33.75,17.93,26.80,10.01,16.78,32.61,48.02,19.01,29.01,0
4,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,64.24,12.27,16.97,6.71,10.25,26.04,39.90,10.65,29.25,0


In [25]:
df_soil_val_weekly = pd.merge(df_soil, df_val_weekly, on=['fips'])
df_soil_val_weekly.head() 

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,1.70,19.13,24.95,12.18,12.75,34.70,43.79,21.97,21.81,2
1,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,107.03,13.47,20.54,8.91,11.62,27.61,40.82,16.42,24.44,2
2,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,106.58,18.44,30.28,10.16,20.12,35.75,56.02,20.16,35.85,1
3,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,53.88,17.73,26.06,11.31,14.74,36.09,48.33,21.10,27.24,1
4,1001,32.536382,-86.64449,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.017,...,75.63,15.04,22.15,9.32,12.84,30.03,42.81,18.52,24.28,1


In [26]:
%%time
# saving df to use for feature selection, model and data viusalization 
df_soil_train_weekly.to_csv('weekly_train.csv')
df_soil_test_weekly.to_csv('weekly_test.csv')
df_soil_val_weekly.to_csv('weekly_val.csv')

CPU times: user 2min 31s, sys: 14 s, total: 2min 45s
Wall time: 2min 55s
